In [1]:
import pandas as pd
import numpy as np
import fastf1 as ff1
from datetime import datetime as dt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [41]:
df = pd.read_csv("dataframe_for_neuralnetwork.csv")

In [42]:
df.head()

,Unnamed: 0,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y,NameEncoder
0,0,2,1,Max Verstappen,1.0,1.0,1.0,1,26.0,0.070413,1.0,1.0,-0.062816,1.197880,-0.532516,-0.436472,0.027647,3,-0.900151,0,1.0,13
1,1,2,1,Max Verstappen,1.0,1.0,1.0,1,26.0,0.058025,2.0,1.0,0.413454,1.256385,-0.519926,-0.490961,-0.021169,3,-0.816672,0,1.0,13
2,2,2,1,Max Verstappen,1.0,1.0,1.0,1,26.0,0.063755,3.0,1.0,0.426054,1.301560,-0.511492,-0.518206,-0.143208,3,-0.733194,0,1.0,13
3,3,2,1,Max Verstappen,1.0,1.0,1.0,1,26.0,0.062426,4.0,1.0,0.415731,1.290600,-0.507091,-0.463717,0.100871,3,-0.649715,0,1.0,13
4,4,2,1,Max Verstappen,1.0,1.0,1.0,1,26.0,0.069021,5.0,1.0,0.464919,1.314890,-0.502447,-0.518206,-0.094392,3,-0.566236,0,1.0,13


In [43]:
encoder_combination = df[["FullName", "NameEncoder"]].drop_duplicates()

In [44]:
encoder_combination

,FullName,NameEncoder
0,Max Verstappen,13
747,Sergio Perez,18
1451,Carlos Sainz,1
2185,Charles Leclerc,2
2956,George Russell,6
3736,Lando Norris,10
4530,Lewis Hamilton,11
5288,Oscar Piastri,16
6089,Fernando Alonso,5
6885,Lance Stroll,9


In [45]:
df = df.drop(columns=["Unnamed: 0", "EventName", "FullName", "Points", "LapNumber", "ClassifiedPosition", "Position_y"])

In [46]:
df["Position_x"] = np.where(df["Position_x"] == 1.0, 1, 0)

In [47]:
df

,Session5DateUtc,Position_x,GridPosition,Status,LapTime,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,NameEncoder
0,1,1,1.0,1,0.070413,1.0,-0.062816,1.197880,-0.532516,-0.436472,0.027647,3,-0.900151,0,13
1,1,1,1.0,1,0.058025,1.0,0.413454,1.256385,-0.519926,-0.490961,-0.021169,3,-0.816672,0,13
2,1,1,1.0,1,0.063755,1.0,0.426054,1.301560,-0.511492,-0.518206,-0.143208,3,-0.733194,0,13
3,1,1,1.0,1,0.062426,1.0,0.415731,1.290600,-0.507091,-0.463717,0.100871,3,-0.649715,0,13
4,1,1,1.0,1,0.069021,1.0,0.464919,1.314890,-0.502447,-0.518206,-0.094392,3,-0.566236,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191031,1,0,0.0,0,0.125421,2.0,2.992176,-0.440264,-0.538994,0.789534,-0.338472,2,-1.150587,1,17
191032,1,0,0.0,0,-0.103098,2.0,0.232946,-0.458778,-0.529827,0.707800,-0.289656,2,-1.067108,1,17
191033,1,0,0.0,0,-0.104428,2.0,0.237045,-0.463962,-0.541805,0.735045,-0.216432,2,-0.983629,1,17
191034,1,0,0.0,0,-0.102622,2.0,0.221560,-0.444708,-0.527626,-0.226092,-0.192024,2,-0.900151,1,17


In [48]:
X = df.drop(columns=["Position_x"])
y = df["Position_x"]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [50]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X, y, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


c:\Users\Caio\Desktop\apps\Project F1\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5970/5970 ━━━━━━━━━━━━━━━━━━━━ 6s 843us/step - accuracy: 0.9506 - loss: 0.1263 - val_accuracy: 0.9672 - val_loss: 0.0748
Epoch 2/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 828us/step - accuracy: 0.9675 - loss: 0.0700 - val_accuracy: 0.9743 - val_loss: 0.0510
Epoch 3/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 811us/step - accuracy: 0.9733 - loss: 0.0539 - val_accuracy: 0.9762 - val_loss: 0.0453
Epoch 4/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 813us/step - accuracy: 0.9764 - loss: 0.0475 - val_accuracy: 0.9762 - val_loss: 0.0432
Epoch 5/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 811us/step - accuracy: 0.9793 - loss: 0.0424 - val_accuracy: 0.9822 - val_loss: 0.0371
Epoch 6/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 811us/step - accuracy: 0.9795 - loss: 0.0403 - val_accuracy: 0.9814 - val_loss: 0.0331
Epoch 7/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 818us/step - accuracy: 0.9811 - loss: 0.0373 - val_accuracy: 0.9832 - val_loss: 0.0309
Epoch 8/10
5970/5970 ━━━━━━━━━━━━━━━━━━━━ 5s 817us/step - accuracy: 0.9819 - loss: 0.03

In [51]:
loss, accuracy = model.evaluate(X_test, y_test)
print({accuracy})

1791/1791 ━━━━━━━━━━━━━━━━━━━━ 1s 505us/step - accuracy: 0.9832 - loss: 0.0329
{0.9835633635520935}


In [ ]:
def mean_features(df):
    df["AverageGridPosition"] = df.groupby('FullName')["GridPosition"].transform('mean')
    df["AverageStatus"] = df.groupby('FullName')["Status"].transform('mean')
    df["AverageStatus"] = df.groupby('FullName')["Status"].transform('mean')